In [72]:
import pandas as pd, numpy as np
import xml.etree.ElementTree as et 

In [73]:
path = '../../../../MyDocuments/Logs/'

In [74]:
tree = et.parse(path + '/Run/2020-04-25-083310.gpx')
root = tree.getroot()
root.tag
root.attrib

{'version': '1.1',
 'creator': 'Runkeeper - http://www.runkeeper.com',
 '{http://www.w3.org/2001/XMLSchema-instance}schemaLocation': 'http://www.topografix.com/GPX/1/1 http://www.topografix.com/GPX/1/1/gpx.xsd'}

In [57]:
track_points = []
ns = {'trkpt': 'http://www.topografix.com/GPX/1/1'}
row_id = 0
for child in root:
    if 'trk' in child.tag:
        new_root = child
for child in new_root:
    if 'trkseg' in child.tag:
        new_root = child
for child in new_root:
    latitude = child.get('lat')
    longitude = child.get('lon')
    elevation = child.find('trkpt:ele', ns).text
    time = child.find('trkpt:time', ns).text
    track_point = {'Latitude': latitude, 
                   'Longitude': longitude, 
                   'Elevation': elevation, 
                   'Time': time}
    track_points.append(track_point)
    row_id += 1

In [58]:
df = pd.DataFrame(track_points)
df['Time'] = pd.to_datetime(df['Time'])
df['Date'] = df['Time'].dt.date
df['Elevation'] = pd.to_numeric(df['Elevation'])
df['Latitude'] = df['Latitude'].astype(float)
df['Longitude'] = df['Longitude'].astype(float)
df = df.sort_values(by = ['Time'], ascending = [True]).reset_index(drop=True)

In [67]:
def haversine(lat1, lon1, lat2, lon2, to_radians=True, earth_radius=6371):
    """
    slightly modified version: of http://stackoverflow.com/a/29546836/2901002

    Calculate the great circle distance between two points
    on the earth (specified in decimal degrees or in radians)

    All (lat, lon) coordinates must have numeric dtypes and be of equal length.

    """
    if to_radians:
        lat1, lon1, lat2, lon2 = np.radians([lat1, lon1, lat2, lon2])

    a = np.sin((lat2-lat1)/2.0)**2 + \
        np.cos(lat1) * np.cos(lat2) * np.sin((lon2-lon1)/2.0)**2

    return earth_radius * 2 * np.arcsin(np.sqrt(a)) * 1000

In [68]:
df['Distance'] = haversine(df['Latitude'].shift()
                           , df['Longitude'].shift()
                           , df['Latitude']
                           , df['Longitude'])
df['Distance'].fillna(0, inplace=True)

In [69]:
df['ElevationGain'] = df['Elevation'] - df['Elevation'].shift()
df['ElevationGain'].fillna(0, inplace=True)

In [70]:
total_elevation_gain = sum(df.loc[df['ElevationGain']>0, 'ElevationGain'])
total_elevation_lost = sum(df.loc[df['ElevationGain']<0, 'ElevationGain']) * -1
total_time = pd.Timedelta(max(df['Time']) - min(df['Time'])).seconds / 60
total_distance = sum(df['Distance'])
activity_date = min(df['Date'])
activity_time = min(df['Time'])

In [71]:
total_distance

4750.209065175834

In [75]:
df.head()

,Elevation,Latitude,Longitude,Time,Date,Distance,ElevationGain
0,4.8,51.487701,-0.006794,2020-04-25 08:33:10+00:00,2020-04-25,0.000000,0.0
1,4.9,51.487583,-0.006991,2020-04-25 08:33:26+00:00,2020-04-25,18.926537,0.1
2,4.9,51.487560,-0.007009,2020-04-25 08:33:27+00:00,2020-04-25,2.844996,0.0
3,4.9,51.487537,-0.007014,2020-04-25 08:33:28+00:00,2020-04-25,2.580809,0.0
4,4.9,51.487518,-0.007011,2020-04-25 08:33:29+00:00,2020-04-25,2.122890,0.0


In [77]:
out_path = path + 'RunCSV\\' + pd.to_datetime(activity_time).strftime('%Y-%m-%d-%H-%M-%S') + '.csv'
df.to_csv(out_path, index = False)